In [ ]:
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!wget -c "https://figshare.com/ndownloader/files/10798046"

In [ ]:
!gzip -d GoogleNews-vectors-negative300.bin.gz

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip -q glove.6B.zip

In [ ]:
#Fine Tune Glove
# https://gist.github.com/chmodsss/867e01cc3eeeaa42226ac931709077dc
# https://towardsdatascience.com/fine-tune-glove-embeddings-using-mittens-89b5f3fe4c39
# https://stackoverflow.com/questions/50909726/fine-tuning-glove-embeddings

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, time, string
import numpy as np
import pandas as pd
import time, os
import string
import nltk
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn import metrics
from numpy import quantile, where, random
import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from sklearn.utils import resample
import warnings
warnings.filterwarnings('ignore')

messages = pd.read_csv("super23.csv", encoding='latin-1')
# messages = pd.read_excel("punny.xlsx")
# messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3"], axis = 1)
messages.columns = ["label","message"]
print (messages.head())

In [ ]:
url = "GoogleNews-vectors-negative300.bin"
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map
embeddings = read_glove_vector('glove.6B.100d.txt') #glove.6B.50d.txt

In [ ]:
print (embeddings["win"])

In [ ]:
import time
start = time.time()
# messages_train = messages.loc[0:55460, :]
messages_train = messages.loc[37616:65736, :] #0:42627, 10658:53286, 0:52812
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

def random_undersampling(tmp_df, TARGET_LABEL):
    df_majority = tmp_df[tmp_df[TARGET_LABEL] == 1]
    df_minority = tmp_df[tmp_df[TARGET_LABEL] == 0]

    # Downsample majority class
    df_majority_downsampled = resample(df_majority,
                                       replace=False,              # sample without replacement
                                       n_samples=len(df_minority), # to match minority class
                                       random_state=None)        # reproducible results
    # Combine minority class with downsampled majority class
    df_downsampled = pd.concat([df_majority_downsampled, df_minority])

    print("Undersampling complete!")
    print(df_downsampled[TARGET_LABEL].value_counts())
    return df_downsampled

df_downsampled = random_undersampling(messages_train, 'label')
df_downsampled = df_downsampled.sample(frac=1) #Shuffle the data
df_downsampled = df_downsampled.reset_index() #Reset the index
df_downsampled = df_downsampled.drop(columns=['index']) # Drop original index col

df_downsampled.head()

'''
Lets make some negatives out of the positives by unlabeling a certain number of data points

'''
# Make a new df because we will need that for later
df = df_downsampled.copy()

docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('english') # removing stop words

for doc in df['message'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    try:
        for word in doc.split(' '): # looping through each word of a single document and spliting through space
            if word not in stopwords: # if word is not present in stopwords then (try)
                try:
                    word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                    temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
                except:
                    pass
        doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
        docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
    except:
        pass
print (docs_vectors.shape)
stop = time.time()
print(f"Trg time: {stop - start}s")  

docs_vectors['label'] = df['label']
print (docs_vectors.shape)
docs_vectors = docs_vectors.replace(np.nan, 0) # All data frame
# docs_vectors = docs_vectors.dropna()
train_lbl = docs_vectors['label']
print (train_lbl)

# docs_vectors = docs_vectors.dropna()
# train_lbl = train_lbl.dropna()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

# model = SVC(gamma='scale', random_state=1)
# model.fit(docs_vectors, train_lbl)
# test_pred = model.predict(docs_vectors_test)


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from baggingPU import BaggingClassifierPU
from sklearn.ensemble import RandomForestClassifier

print('Training bagging classifier...')
pu_start = time.perf_counter()
# Bagging with RandomForestClassifier
bc = BaggingClassifierPU(RandomForestClassifier(n_estimators=20, random_state=3),
                         n_estimators = 50,
                         n_jobs = -1,
                         max_samples = sum(train_lbl)  # Each training sample will be balanced
                        )
docs_vectors = docs_vectors.iloc[:,:-1]
bc.fit(docs_vectors, train_lbl)
pu_end = time.perf_counter()
print('Done!')
print('Trg Time:', pu_end - pu_start)

In [ ]:
import time
start = time.time()
docs_vectors_test = pd.DataFrame() # creating empty final dataframe
messages_test = messages.loc[0:37615, :] # 42627:53286, 0:10657, 52812:53286
#For Test Data
docs_vectors_test = pd.DataFrame() # creating empty final dataframe
for doc in messages_test['message'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp1 = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    try:
        for word in doc.split(' '): # looping through each word of a single document and spliting through space
            if word not in stopwords: # if word is not present in stopwords then (try)
                try:
                    word_vec1 = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                    temp1 = temp1.append(pd.Series(word_vec1), ignore_index = True) # if word is present then append it to temporary dataframe
                except:
                    pass
        doc_vector1 = temp1.mean() # take the average of each column(w0, w1, w2,........w300)
        docs_vectors_test = docs_vectors_test.append(doc_vector1, ignore_index = True) # append each document value to the final dataframe
    except:
        pass
print (docs_vectors_test.shape)
stop = time.time()
print(f"Testing time: {stop - start}s")  

(37616, 100)
Testing time: 442.5656087398529s


In [ ]:
messages_test['label'].to_csv("lblpu212.csv", index = False)
ml = pd.read_csv("lblpu212.csv", encoding='latin-1')
ml.columns = ["label"]
print (ml.head())
# messages = pd.read_excel("punny.xlsx")
# messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3"], axis = 1)

   label
0      1
1      1
2      1
3      1
4      1


In [ ]:
import time
start = time.time()

test_lbl = ml['label']
docs_vectors_test['label'] = ml['label']
docs_vectors_test = docs_vectors_test.dropna()

print('---- {} ----'.format('PU Bagging'))
print('')

# docs_vectors_test = docs_vectors_test.iloc[:,:-1]
test_pred = bc.predict(docs_vectors_test.drop('label', axis = 1))
stop = time.time()
print(f"Testing time: {stop - start}s") 

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
print(confusion_matrix(docs_vectors_test['label'], test_pred))
print("Precision: ", precision_score(docs_vectors_test['label'], test_pred))
print("Recall: ", recall_score(docs_vectors_test['label'], test_pred))
print ("Accuracy: ", accuracy_score(docs_vectors_test['label'], test_pred))

---- PU Bagging ----

Testing time: 8.610342979431152s
[[33842  2703]
 [  137   553]]
Precision:  0.16984029484029484
Recall:  0.8014492753623188
Accuracy:  0.9237276755740567
